# Task 4

**Given task:**
Implement a working A* (Astar) function using your previously created priority queue code.

Here are some publications:

- [Breadth-first search](https://www.geeksforgeeks.org/artificial-intelligence/strips-in-ai/) – Wikipedia, searches evenly in all directions  
- [Dijkstra's algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm) – Wikipedia, takes movement weights into account  
- [A* search algorithm](https://en.wikipedia.org/wiki/A*_search_algorithm) – Wikipedia, searches while prioritizing the direction of the goal  
- [Astar pseudocode](https://www.gamedev.net/tutorials/programming/artificial-intelligence/a-pathfinding-for-beginners-r2003/) - Pseudocode  

The code is flexible and not tied to a specific task. Graph-based representation for pathfinding is done.

In [48]:
class PriorityQueue:
    def __init__(self):
        self.Queue = []
    
    def Enqueue(self, value, priority):
        # Use insertion sort: find correct position and insert
        # Higher priority values come first (descending order)
        # For A*, we want LOWEST f-score first, so we negate priority
        position = 0
        while position < len(self.Queue) and self.Queue[position][0] >= priority:
            position = position + 1
        self.Queue.insert(position, (priority, value))
    
    def Dequeue(self):
        # Remove and return the first item (highest priority)
        if len(self.Queue) == 0:
            raise Exception("Priority queue is empty")
        item = self.Queue.pop(0)
        return item[1]
    
    def Peek(self):
        # View the highest priority item without removing it
        if len(self.Queue) == 0:
            raise Exception("Priority queue is empty")
        return self.Queue[0][1]
    
    def IsEmpty(self):
        return len(self.Queue) == 0
    
    def Contains(self, value):
        for item in self.Queue:
            if item[1] == value:
                return True
        return False
    
    def Remove(self, value):
        # Remove a specific value from the queue
        for i, item in enumerate(self.Queue):
            if item[1] == value:
                self.Queue.pop(i)
                return True
        return False

In [49]:
def AStar(start, goal, get_neighbors, heuristic, get_cost=None):
    """
    A* pathfinding algorithm - finds the optimal path from start to goal.
    
    Parameters:
    -----------
    start : any
        The starting node
    goal : any
        The goal/target node
    get_neighbors : function(node) -> list
        Function that returns a list of neighbor nodes for a given node.
        Can return list of nodes or list of (node, cost) tuples.
    heuristic : function(node, goal) -> number
        Heuristic function estimating cost from node to goal.
        Must be admissible (never overestimate) for optimal results.
    get_cost : function(from_node, to_node) -> number, optional
        Function returning the cost to move from one node to another.
        If None, assumes cost of 1 for each move.
    
    Returns:
    --------
    tuple: (path, total_cost) where path is list of nodes from start to goal,
           or (None, None) if no path exists.
    """
    
    # Default cost function: cost is 1 for each move
    def default_cost(a, b):
        return 1
    
    if get_cost is None:
        get_cost = default_cost
    
    # OPEN list: nodes to be evaluated (using priority queue)
    # We use negative f-score as priority because our queue gives highest priority first
    open_list = PriorityQueue()
    
    # CLOSED list: nodes already evaluated
    closed_set = set()
    
    # Track which nodes are in open list for quick lookup
    open_set = set()
    
    # g_score: cost from start to each node
    g_score = {start: 0}
    
    # f_score: g_score + heuristic estimate
    f_score = {start: heuristic(start, goal)}
    
    # Track the path: parent of each node
    came_from = {}
    
    # Initialize: put start node in OPEN list
    # Use negative f_score because our queue dequeues highest priority first
    open_list.Enqueue(start, -f_score[start])
    open_set.add(start)
    
    while not open_list.IsEmpty():
        # Get node with lowest f_score (highest priority = most negative)
        current = open_list.Dequeue()
        open_set.discard(current)
        
        # Check if we reached the goal
        if current == goal:
            # Reconstruct and return the path
            path = reconstruct_path(came_from, current)
            return path, g_score[current]
        
        # Add current to CLOSED list
        closed_set.add(current)
        
        # Get neighbors of current node
        neighbors = get_neighbors(current)
        
        for neighbor_data in neighbors:
            # Handle both formats: 
            # - (node, cost) tuple where node can be any type
            # - just node (cost determined by get_cost function)
            # We check if it looks like (node, numeric_cost) tuple
            if (isinstance(neighbor_data, tuple) and len(neighbor_data) == 2 
                and isinstance(neighbor_data[1], (int, float)) 
                and not isinstance(neighbor_data[0], (int, float))):
                # This is (node, cost) format where node is not a number
                neighbor, edge_cost = neighbor_data
            elif (isinstance(neighbor_data, tuple) and len(neighbor_data) == 2 
                  and isinstance(neighbor_data[0], tuple)):
                # This is ((x,y), cost) format - grid with costs
                neighbor, edge_cost = neighbor_data
            else:
                # This is just a node (could be tuple like (x,y) or any other type)
                neighbor = neighbor_data
                edge_cost = get_cost(current, neighbor)
            
            # Skip if already evaluated
            if neighbor in closed_set:
                continue
            
            # Calculate tentative g_score
            tentative_g = g_score[current] + edge_cost
            
            # Check if this path to neighbor is better
            if neighbor in open_set:
                if tentative_g >= g_score.get(neighbor, float('inf')):
                    continue  # Not a better path
                # Remove old entry to update with better path
                open_list.Remove(neighbor)
                open_set.discard(neighbor)
            
            # This is the best path to neighbor so far
            came_from[neighbor] = current
            g_score[neighbor] = tentative_g
            f_score[neighbor] = tentative_g + heuristic(neighbor, goal)
            
            # Add to OPEN list
            open_list.Enqueue(neighbor, -f_score[neighbor])
            open_set.add(neighbor)
    
    # No path found
    return None, None

In [50]:
def reconstruct_path(came_from, current):
    """
    Reconstruct the path from start to current by following parent links.
    """
    path = [current]
    while current in came_from:
        current = came_from[current]
        path.append(current)
    path.reverse()
    return path

In [51]:
# Grid based path finding (2-D grid)
def create_grid_functions(width, height, obstacles=None):
    """
    Create neighbor and heuristic functions for a 2D grid.
    
    Parameters:
    -----------
    width, height : int
        Grid dimensions
    obstacles : set, optional
        Set of (x, y) positions that are blocked
    
    Returns:
    --------
    tuple: (get_neighbors, heuristic) functions
    """
    if obstacles is None:
        obstacles = set()
    
    def get_neighbors(pos):
        """Return walkable neighbors (4-directional movement)."""
        x, y = pos
        neighbors = []
        # 4 directions: up, down, left, right
        for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < width and 0 <= ny < height:
                if (nx, ny) not in obstacles:
                    neighbors.append((nx, ny))
        return neighbors
    
    def manhattan_heuristic(pos, goal_pos):
        """Manhattan distance heuristic."""
        return abs(pos[0] - goal_pos[0]) + abs(pos[1] - goal_pos[1])
    
    return get_neighbors, manhattan_heuristic

In [52]:
# Optimal path finding for nodes and trees
def create_graph_functions(graph, heuristic_values=None):
    """
    Create neighbor and heuristic functions for a weighted graph.
    
    Parameters:
    -----------
    graph : dict
        Dictionary mapping node -> list of (neighbor, cost) tuples
        Example: {'A': [('B', 4), ('C', 2)], 'B': [('D', 3)], ...}
    heuristic_values : dict, optional
        Dictionary mapping node -> estimated cost to goal
        If None, uses 0 (becomes Dijkstra's algorithm)
    
    Returns:
    --------
    tuple: (get_neighbors, heuristic) functions
    """
    if heuristic_values is None:
        heuristic_values = {}
    
    def get_neighbors(node):
        """Return neighbors with their edge costs."""
        return graph.get(node, [])
    
    def graph_heuristic(node, goal):
        """Return estimated cost to goal."""
        return heuristic_values.get(node, 0)
    
    return get_neighbors, graph_heuristic

In [53]:
if __name__ == "__main__":
    # Test 1: Grid pathfinding
    print("\n Test 1: Grid Pathfinding")
    print("Grid 5x5 with obstacles at (1,1), (1,2), (2,1)")
    print("Finding path from (0,0) to (4,4)")
    
    obstacles = {(1, 1), (1, 2), (2, 1), (0, 2)}
    get_neighbors, heuristic = create_grid_functions(5, 5, obstacles)
    
    path, cost = AStar((0, 0), (4, 4), get_neighbors, heuristic)
    
    if path:
        print(f"Path found: {path}")
        print(f"Total cost: {cost}")
        
        # Visualize the grid
        print("\nGrid visualization (S=start, G=goal, *=path, #=obstacle):")
        for y in range(4, -1, -1):
            row = ""
            for x in range(5):
                if (x, y) == (0, 0):
                    row += "S "
                elif (x, y) == (4, 4):
                    row += "G "
                elif (x, y) in obstacles:
                    row += "# "
                elif (x, y) in path:
                    row += "* "
                else:
                    row += ". "
            print(row)
    else:
        print("No path found!")
    
    # Test 2: Weighted graph
    print("\n -------------------------------------")
    print("\n Test 2: Weighted Graph")
    print("Graph: A->B(4), A->C(2), B->D(3), C->B(1), C->D(5)")

    # Design a net:
    graph = {
        'A': [('B', 4), ('C', 2)],
        'B': [('D', 3)],
        'C': [('B', 1), ('D', 5)],
        'D': []
    }
    # Admissible heuristic estimates to D (must never overestimate)
    # Actual costs: A->D = 6 (A->C->B->D), B->D = 3, C->D = 4 (C->B->D)
    h_values = {'A': 5, 'B': 3, 'C': 3, 'D': 0}
    
    get_neighbors, heuristic = create_graph_functions(graph, h_values)
    
    path, cost = AStar('A', 'D', get_neighbors, heuristic)
    
    if path:
        print(f"Path found: {' -> '.join(path)}")
        print(f"Total cost: {cost}")
    else:
        print("No path found!")
    
    # Test 3: No path exists
    print("\n -------------------------------------")
    print("\n Test 3: No Path Exists")
    print("Grid 3x3 with obstacle blocking all paths")
    
    # Block the middle completely
    obstacles = {(1, 0), (1, 1), (1, 2)}
    get_neighbors, heuristic = create_grid_functions(3, 3, obstacles)
    
    path, cost = AStar((0, 0), (2, 2), get_neighbors, heuristic)
    
    if path:
        print(f"Path found: {path}")
    else:
        print("No path found (as expected)")


 Test 1: Grid Pathfinding
Grid 5x5 with obstacles at (1,1), (1,2), (2,1)
Finding path from (0,0) to (4,4)
Path found: [(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (4, 4)]
Total cost: 8

Grid visualization (S=start, G=goal, *=path, #=obstacle):
. . . * G 
. . . * . 
# # . * . 
. # # * . 
S * * * . 

 -------------------------------------

 Test 2: Weighted Graph
Graph: A->B(4), A->C(2), B->D(3), C->B(1), C->D(5)
Path found: A -> C -> B -> D
Total cost: 6

 -------------------------------------

 Test 3: No Path Exists
Grid 3x3 with obstacle blocking all paths
No path found (as expected)
